In [41]:
import sys
import os


sys.path.append(os.path.abspath(".."))


In [42]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

import matplotlib.pyplot as plt
import os
from datetime import datetime

# Import your environment
from env.stardew_mine_env import StardewMineEnv

In [43]:
def make_env():
    env = StardewMineEnv(size=10, max_floor=10, max_energy=100, local_view_size=5)
    env = Monitor(env)  # Logs episode rewards
    return env

env = DummyVecEnv([make_env])


In [44]:
import os
from datetime import datetime


base_dir = os.getcwd()
parent_dir = os.path.dirname(base_dir)

models_dir = os.path.join(parent_dir, "models")
os.makedirs(models_dir, exist_ok=True) 

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"ppo_miningbot_{timestamp}.zip"
save_path = os.path.join(models_dir, model_filename)

print("Full save path for model:", save_path)



Full save path for model: /Users/alejandroolivares-lopez/Desktop/Strawdew_Valley_Mining_Bot/models/ppo_miningbot_20251130_154045.zip


In [45]:
model = PPO(
    policy="MultiInputPolicy",
    env=env,
    verbose=1,
    learning_rate=3e-4,
    batch_size=64,
    n_steps=2048,
    gamma=0.99,
    tensorboard_log=f"logs/{run_name}"
)



Using cpu device


In [46]:
# ENSURE MODELS FOLDER EXISTS
model_dir = "../models"
os.makedirs(model_dir, exist_ok=True)

# TRAIN
TIMESTEPS = 200_000
model.learn(total_timesteps=TIMESTEPS)

# SAVE CORRECTLY — DO NOT ADD ".zip"
model_path = os.path.join(model_dir, "ppo_miningbot")
model.save(model_path)

print("Training complete!")
print("Saved to:", model_path + ".zip")



Logging to logs/ppo_miningbot_20251129_151600/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 212      |
|    ep_rew_mean     | -102     |
| time/              |          |
|    fps             | 5294     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 212      |
|    ep_rew_mean     | -102     |
| time/              |          |
|    fps             | 5294     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 233         |
|    ep_rew_mean          | -106        |
| time/                   |             |
|    fps                  | 3697        |
|    iterations 

In [47]:
model = PPO.load(f"{save_path}/ppo_miningbot")

test_env = StardewMineEnv()
obs, _ = test_env.reset()

total_reward = 0
done = False

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = test_env.step(action)
    total_reward += reward

print("Test episode reward:", total_reward)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alejandroolivares-lopez/Desktop/Strawdew_Valley_Mining_Bot/models/ppo_miningbot_20251130_154045.zip/ppo_miningbot.zip'

In [ ]:
import pandas as pd

monitor_file = [f for f in os.listdir(".") if f.endswith(".monitor.csv")]
monitor_file

In [ ]:
df = pd.read_csv(monitor_file[0], skiprows=1)

plt.plot(df["r"])  # r = reward
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("PPO Training Reward Curve")
plt.show()
